In [37]:
# 读数据
import pandas as pd
df_train=pd.read_excel("../data/df_train.xlsx",index_col=0)
df_test=pd.read_excel("../data/df_test.xlsx",index_col=0)

In [38]:
x=[]
y=[]
import ast
from tqdm import tqdm
for i in tqdm(range(len(df_train))):
    x.append(df_train.loc[i]['text'])
    if df_train.loc[i]['label']=='[0, 0, 0, 0, 0, 0]':
        y.append(0)
    else:
#         print(df_train.loc[i]['label'])
        y.append(1)

for i in tqdm(range(len(df_test))):
    x.append(df_train.loc[i]['text'])
    if df_test.loc[i]['label']=='[0, 0, 0, 0, 0, 0]':
        y.append(0)
    else:
#         print(df_test.loc[i]['label'])
        y.append(1)

100%|██████████| 6680/6680 [00:01<00:00, 5990.90it/s]


In [39]:
print(list(set(y)))

[0, 1]


In [40]:
data_=[list(t) for t in zip(x,y)]

In [41]:
import random
dataSet0=[]
trainDataSet = []
testDataSet=[]
dataSet0 = data_
proportion = 0.2
dataNumber = len(dataSet0)  # 数据集数据条数
print(dataNumber)
testNumber = int(dataNumber * proportion)  # 测试集数据条数
testDataSet = random.sample(dataSet0, testNumber)  # 测试集
for i in testDataSet:  # 将已经选定的测试集数据从数据集中删除
    if i in dataSet0:
        dataSet0.remove(i)
trainDataSet=dataSet0
#获取trainDataSet，testDataSet
print(len(trainDataSet))
print(len(testDataSet))

#train.columns = ['text', 'labels']train_df, valid_df = 
#train_test_split(train, test_size=0.2, stratify=train[‘labels’], random_state=42)


36782
29426
7356


In [46]:

from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Preparing train data
'''[
    ["Aragorn was the heir of Isildur", 1],
    ["Frodo was the heir of Isildur", 0],
]'''
train_data=trainDataSet
train_df = pd.DataFrame(train_data)
train_df.columns = ["text", "labels"]

# Preparing eval data
'''eval_data = [
     ["Theoden was the king of Rohan", 1],
     ["Merry was the king of Rohan", 0],
# ]'''
eval_data=testDataSet
eval_df = pd.DataFrame(eval_data)
eval_df.columns = ["text", "labels"]

# Optional model configuration
model_args = ClassificationArgs(num_train_epochs=20,overwrite_output_dir=True)

# Create a ClassificationModel
model = ClassificationModel(
    "bert", "bert-base-chinese", args=model_args ,use_cuda=True
)
     
# Train the model
model.train_model(train_df)



Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

  0%|          | 0/29426 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_bert_128_2_2


Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Running Epoch 0 of 20:   0%|          | 0/3679 [00:00<?, ?it/s]

Running Epoch 1 of 20:   0%|          | 0/3679 [00:00<?, ?it/s]

Running Epoch 2 of 20:   0%|          | 0/3679 [00:00<?, ?it/s]

RuntimeError: [enforce fail at inline_container.cc:300] . unexpected pos 240741440 vs 240741328

In [45]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df)
# print(model_outputs)
#print(result)
#print(wrong_predictions)

outputs=[]
for i in model_outputs:
    if i[0]>i[1]:
        outputs.append(0)
    else:
        outputs.append(1)
# print(outputs)

y_test=[]
for i in testDataSet:
    y_test.append(i[1])
from sklearn import metrics
print(metrics.classification_report(y_test, outputs))
print("准确率:", metrics.accuracy_score(y_test, outputs))

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/7356 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_2_2


Running Evaluation:   0%|          | 0/920 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.2953471062512684, 'tp': 1991, 'tn': 2805, 'fp': 1272, 'fn': 1288, 'auroc': 0.7040388202610574, 'auprc': 0.640827241397852, 'eval_loss': 0.6243495277736498}


              precision    recall  f1-score   support

           0       0.69      0.69      0.69      4077
           1       0.61      0.61      0.61      3279

    accuracy                           0.65      7356
   macro avg       0.65      0.65      0.65      7356
weighted avg       0.65      0.65      0.65      7356

准确率: 0.6522566612289288
